In [62]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [63]:
import json
import os
import numpy as np
from utils import *
import math
import heapq

In [64]:
fd_collection = getCollection("team_5_mwdb_phase_2", "fd_collection")
all_images = fd_collection.find()


In [65]:
selected_latent_space = valid_latent_spaces[
    str(input("Enter latent space - one of " + str(list(valid_latent_spaces.keys()))))
]

selected_feature_model = valid_feature_models[
    str(input("Enter feature model - one of " + str(list(valid_feature_models.keys()))))
]

k = int(input("Enter value of k: "))
if k < 1:
    raise ValueError("k should be a positive integer")

selected_dim_reduction_method = str(
    input(
        "Enter dimensionality reduction method - one of "
        + str(list(valid_dim_reduction_methods.keys()))
    )
)

label = int(input("Enter label: "))
if label < 0 and label > 100:
    raise ValueError("k should be between 0 and 100")

knum = int(input("Enter value of knum: "))
if knum < 1:
    raise ValueError("knum should be a positive integer")

label_rep = calculate_label_representatives(fd_collection, label, selected_feature_model)

match selected_latent_space:
    case "":
        if os.path.exists(f"{selected_feature_model}-{selected_dim_reduction_method}-{k}-semantics.json"):
            data = json.load(open(f"{selected_feature_model}-{selected_dim_reduction_method}-{k}-semantics.json"))
        else:
          print(f"{selected_feature_model}-{selected_dim_reduction_method}-{k}-semantics.json does not exist" )
    case "cp":
        if os.path.exists(f"{selected_feature_model}-cp-{k}-semantics.json"):
            data = json.load(open(f"{selected_feature_model}-cp-{k}-semantics.json"))
        else:
          
          print(f"{selected_feature_model}-cp-{k}-semantics.json does not exist" )
    case _:
        if os.path.exists(f"{selected_latent_space}-{selected_feature_model}-{selected_dim_reduction_method}-{k}-semantics.json"):
            data = json.load(open(f"{selected_latent_space}-{selected_feature_model}-{selected_dim_reduction_method}-{k}-semantics.json"))
        else:
          print(f"{selected_latent_space}-{selected_feature_model}-{selected_dim_reduction_method}-{k}-semantics.json does not exist" )


In [66]:
def extract_similarities_ls1(dim_reduction, data, label, label_rep):

  match dim_reduction:

    case 'svd':
      U = np.array(data["image-semantic"])
      S = np.array(data["semantics-core"])
      V = np.transpose(np.array(data["semantic-feature"]))

      comparison_feature_space = np.matmul(U, S)
      comparison_vector = np.matmul(np.matmul(label_rep, V), S)
    
    case "nmf":
      H = np.array(data['semantic-feature'])
      comparison_feature_space = np.array(data['image-semantic'])
      comparison_vector = np.matmul(label_rep, np.transpose(H))

    case ""

  print(comparison_feature_space.shape)
  n = len(comparison_feature_space)
 
  distances = []
  for i in range(n):
    if i != label:
      distances.append({"image_id": i, "label": all_images[i]["true_label"],"distance": math.dist(comparison_vector, comparison_feature_space[i])})

  distances = sorted(distances, key=lambda x: x["distance"], reverse=False)

  similar_labels = []
  unique_labels = set()

  for img in distances:
    if img['label'] not in unique_labels:
      similar_labels.append(img)
      unique_labels.add(img["label"])

      if len(similar_labels) == knum:
        break


  for x in similar_labels:
    print(x)

In [67]:
def extract_similarities_ls3(dim_reduction, data, label):

  match dim_reduction:

    case 'svd':
      U = np.array(data["image-semantic"])
      S = np.array(data["semantics-core"])
      V = np.transpose(np.array(data["semantic-feature"]))

      comparison_feature_space = np.matmul(U, S)
      comparison_vector = comparison_feature_space[label]
    
    case "nmf":
      comparison_feature_space = np.array(data['image-semantic'])
      comparison_vector = comparison_feature_space[label]

  n = len(comparison_feature_space)
  distances = []
  for i in range(n):
    if i != label:
      distances.append({"label": i, "distance": math.dist(comparison_vector, comparison_feature_space[i])})

  distances = sorted(distances, key=lambda x: x["distance"], reverse=False)[:knum]

  for x in distances:
    print(x)

In [68]:
match selected_latent_space:

  case "":
    
    extract_similarities_ls1(selected_dim_reduction_method, data, label, label_rep)

  case "label_sim":

    extract_similarities_ls3(selected_dim_reduction_method, data, label)
    

{'label': 4, 'distance': 0.9931105104385977}
{'label': 92, 'distance': 1.1209182190288185}
{'label': 65, 'distance': 1.2107732156271573}
{'label': 21, 'distance': 1.5053484881391492}
{'label': 2, 'distance': 1.698430977110922}
{'label': 100, 'distance': 1.8636096001573115}
{'label': 95, 'distance': 2.003755992104511}
{'label': 11, 'distance': 2.069066281581252}
{'label': 60, 'distance': 2.070894540798742}
{'label': 88, 'distance': 2.0925931256031}
{'label': 43, 'distance': 2.1056747598887218}
{'label': 33, 'distance': 2.165431005806523}
{'label': 90, 'distance': 2.174626607979455}
{'label': 83, 'distance': 2.188609736988739}
{'label': 68, 'distance': 2.209562202827548}
{'label': 59, 'distance': 2.27130902508622}
{'label': 35, 'distance': 2.276916489521396}
{'label': 70, 'distance': 2.283111150497479}
{'label': 53, 'distance': 2.2871296343421075}
{'label': 42, 'distance': 2.2943393449254192}
{'label': 1, 'distance': 2.299515307388396}
{'label': 89, 'distance': 2.300444335700286}
{'label